In [ ]:
#환경 설정
import os

os.environ['OPENAI_API_KEY'] = '****'
os.environ['LANGCHAIN_API_KEY'] = '****' 
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = '01-04'

In [ ]:
!pip install -qU langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.1 MB/s eta 0:00:00


In [ ]:
# 라이브러리

from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
#질문 examples



examples = [
    {
        "question":"스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer" : """이 질문에 추가 질문이 필요한가요 : 예.
        추가 질문 : 스티브 잡스는 몇 살에 사망했나요?
        중간 답변 : 스티브 잡스는 58세에 사망했습니다.
        추가 질문 : 아인슈타인은 몇 살에 사망했나요?
        중간 답변 : 아인슈타인은 76세에 사망했습니다.
        최종 답변은 : 아인슈타인
        """,
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]


In [ ]:
# 질문의 형태

example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

In [ ]:
example_prompt

PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='Question:\n{question}\nAnswer:\n{answer}')

In [ ]:
# fewshotprompttemplate

prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    suffix = "Question : \n{question} \nAnswer : ",
    input_variables=["question"],
)

In [ ]:
prompt

FewShotPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7d2767a30730>, k=1, example_keys=None, input_keys=None, vectorstore_kwargs=None), example_prompt=PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='Question:\n{question}\nAnswer:\n{answer}'), suffix='Question : \n{question} \nAnswer : ')

In [ ]:
# prompt 생성

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"

final_prompt = prompt.format(question=question)

In [ ]:
# llm 생성

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature=0, #창의성
    model_name="gpt-4o" #모델명
)

In [ ]:
#결과

response = llm.invoke(final_prompt)

'이 질문에 추가 질문이 필요한가요: 예.\n추가 질문: Google은 언제 창립되었나요?\n중간 답변: Google은 1998년에 창립되었습니다.\n추가 질문: Bill Gates는 언제 태어났나요?\n중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.\n추가 질문: 1998년에 Bill Gates는 몇 살이었나요?\n중간 답변: 1998년에 Bill Gates는 43세였습니다.\n최종 답변은: 43세'

In [ ]:
print(response.content)

이 질문에 추가 질문이 필요한가요: 예.
추가 질문: Google은 언제 창립되었나요?
중간 답변: Google은 1998년에 창립되었습니다.
추가 질문: Bill Gates는 언제 태어났나요?
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.
추가 질문: 1998년에 Bill Gates는 몇 살이었나요?
중간 답변: 1998년에 Bill Gates는 43세였습니다.
최종 답변은: 43세


In [ ]:
#예제

question = "Tesla를 인수한 연도에 musk의 나이는 몇 살인가요?"

final_prompt = prompt.format(question=question)

response = llm.invoke(final_prompt)

print(response.content)

이 질문에 추가 질문이 필요한가요: 예.
추가 질문: Tesla를 인수한 연도는 언제인가요?
중간 답변: Elon Musk는 2004년에 Tesla에 투자하고 이사회에 합류했습니다.
추가 질문: Elon Musk는 언제 태어났나요?
중간 답변: Elon Musk는 1971년 6월 28일에 태어났습니다.
추가 질문: 2004년에 Elon Musk의 나이는 몇 살이었나요?
중간 답변: 2004년에 Elon Musk는 33세였습니다.
최종 답변은: 33세


In [ ]:
#예제

question = "openai가 만들어진 연도에 CEO의 나이는 몇 살인가요?"

final_prompt = prompt.format(question=question)

response = llm.invoke(final_prompt)

print(response.content)

이 질문에 추가 질문이 필요한가요: 예.
추가 질문: OpenAI는 언제 만들어졌나요?
중간 답변: OpenAI는 2015년에 만들어졌습니다.
추가 질문: OpenAI의 CEO는 누구인가요?
중간 답변: OpenAI의 CEO는 샘 알트먼입니다.
추가 질문: 샘 알트먼은 언제 태어났나요?
중간 답변: 샘 알트먼은 1985년 4월 22일에 태어났습니다.
추가 질문: 2015년에 샘 알트먼의 나이는 몇 살이었나요?
중간 답변: 2015년에 샘 알트먼은 30세였습니다.
최종 답변은: 30세


In [ ]:
#예제

question = "DeepLearning이 만들어진 연도에 제프리 힌튼의 나이는 몇 살인가요?"

final_prompt = prompt.format(question=question)

response = llm.invoke(final_prompt)

print(response.content)

이 질문에 추가 질문이 필요한가요: 예.
추가 질문: DeepLearning이 만들어진 연도는 언제인가요?
중간 답변: DeepLearning의 개념은 1980년대에 시작되었지만, 2006년에 제프리 힌튼이 딥러닝의 중요한 발전을 이루었습니다.
추가 질문: 제프리 힌튼은 언제 태어났나요?
중간 답변: 제프리 힌튼은 1947년 12월 6일에 태어났습니다.
추가 질문: 2006년에 제프리 힌튼의 나이는 몇 살이었나요?
중간 답변: 2006년에 제프리 힌튼은 59세였습니다.
최종 답변은: 59세


In [ ]:
# chain 생성

question = "엔비디아 만들어진 연도에 젠슨 황 나이는 몇 살인가요?"

chain = prompt | llm

chain.invoke(question)

AIMessage(content='이 질문에 추가 질문이 필요한가요: 예.\n추가 질문: 엔비디아는 언제 설립되었나요?\n중간 답변: 엔비디아는 1993년에 설립되었습니다.\n추가 질문: 젠슨 황은 언제 태어났나요?\n중간 답변: 젠슨 황은 1963년 2월 17일에 태어났습니다.\n추가 질문: 1993년에 젠슨 황의 나이는 몇 살이었나요?\n중간 답변: 1993년에 젠슨 황은 30세였습니다.\n최종 답변은: 30세', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 137, 'prompt_tokens': 591, 'total_tokens': 728, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_703d4ff298', 'finish_reason': 'stop', 'logprobs': None}, id='run-b3f305d1-d55f-4f8a-9f62-84fcd4d0fd08-0', usage_metadata={'input_tokens': 591, 'output_tokens': 137, 'total_tokens': 728, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## 2. Example Selector

### 유사한 벡터를 select 하여 해당 example만 가져와서 사용함

In [ ]:
!pip install -qU langchain_chroma langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.5 MB/s eta 0:00:00


In [ ]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 여기에는 선택 가능한 예시 목록이 있습니다.
    examples,
    # 여기에는 의미적 유사성을 측정하는 데 사용되는 임베딩을 생성하는 임베딩 클래스가 있음
    OpenAIEmbeddings(),
    # 여기에는 임베딩을 저장하고 유사성 검색을 수행하는 데 사용되는 VectorStore 클래스가 있음
    Chroma,
    # 이것은 생성할 예시의 수
    k=1,
)

In [ ]:
# 질문의 형태

example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

In [ ]:
# fewshotprompttemplate

prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    suffix = "Question : \n{question} \nAnswer : ",
    input_variables=["question"],
)

chain = prompt | llm

In [ ]:
question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"

final_prompt = prompt.format(question=question)

In [ ]:
#네이버 예시를 Select 하였음

reslut = chain.invoke(final_prompt)

In [ ]:
print(reslut.content)

이 질문에 추가 질문이 필요한가요: 예.
추가 질문: Google은 언제 창립되었나요?
중간 답변: Google은 1998년에 창립되었습니다.
추가 질문: Bill Gates는 언제 태어났나요?
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.
추가 질문: 1998년에 Bill Gates의 나이는 몇 살이었나요?
중간 답변: 1998년에는 Bill Gates가 43세였습니다.
최종 답변은: 43세
